# 하이브리드 - Hybrid Search


# Hyrbid 검색이란?

하이브리드 검색은 키워드 검색과 Neural Search 검색을 결합하여 검색 관련성을 향상시킵니다. 하이브리드 검색을 구현하려면 검색 시간에 실행되는 [검색 파이프라인](https://opensearch.org/docs/latest/search-plugins/search-pipelines/index/)을 설정해야 합니다. 구성할 검색 파이프라인은 중간 단계에서 검색 결과를 가로채고 [`normalization_processor`](https://opensearch.org/docs/latest/search-plugins/search-pipelines/normalization-processor/)를 적용합니다. `normalization_processor`는 여러 쿼리 절에서의 문서 점수를 정규화하고 결합하여, 선택한 정규화 및 결합 기법에 따라 문서를 재점수화합니다.


# 사전 준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다. 단 여기서는 코드를 간결하게 하기 위해 utils.py에 정의한 keyword_search와 semantic_search 함수를 재사용합니다.


필요한 패키지를 설치합니다


In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

### OpenSearch 도메인에 연결


CloudFormation 스택에서 필요한 정보를 가져오기 위한 함수를 선언합니다.


In [ ]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

OpenSearch 도메인에 접속하기 위한 호스트 정보와 인증정보를 가져옵니다


In [ ]:
import boto3, json
import pandas as pd
from utils import get_cfn_outputs, keyword_search, semantic_search

region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

위에서 가져온 정보를 바탕으로 opensearch-py 패키지를 사용하여 OpenSearch 도메인에 접속합니다.


In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

모델 아이디와 인덱스명을 설정하고 이 과정에서 사용할 인덱스가 잘 준비되어 있는지 확인합니다.


In [ ]:
model_id = "jK-l048BhWoFUAg8WOdO"
index_name = "movie_semantic"

count = aos_client.count(index=index_name)
print(count)

# 검색 함수 정의하기


## 키워드 함수 정의


In [ ]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["plot"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=["_score", "title", "plot", "genre", "rating", "main_act"],
    )
    display(query_result_df)

## 시맨틱 검색 함수 정의


In [ ]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

## 하이브리드 검색 함수 정의

아래와 같이 Hybrid 검색 함수를 정의합니다. 중요한 부분은 다음과 같습니다.

1. **`query`** 부분에서 **`hybrid`** 쿼리를 사용하여 텍스트 기반 검색과 벡터 기반 검색을 결합합니다.
    - **`multi_match`** 쿼리는 **`title`**, **`text`**, **`genre`** 필드에서 **`query_text`**와 일치하는 문서를 찾습니다.
    - **`neural`** 쿼리는 **`vector_field`**에 저장된 벡터 데이터와 **`query_text`**를 사용하여 유사도 기반 검색을 수행합니다. **`model_id`**는 사용할 모델을 지정하고, **`k`**는 반환할 최대 문서 수를 지정합니다.
2. **`search_pipeline`**은 텍스트 기반 검색과 벡터 기반 검색의 결과를 결합하는 방식을 지정합니다.
    - **`normalization-processor`**는 두 검색 결과의 점수를 정규화합니다.
    - **`combination`** 부분에서 **`arithmetic_mean`** 기법을 사용하여 두 검색 결과의 점수를 가중 평균합니다. 여기서는 텍스트 기반 검색 결과에 0.3의 가중치를, 벡터 기반 검색 결과에 0.7의 가중치를 부여합니다.


In [ ]:
def hybrid_search(query_text, keyword_weight=0.3, semantic_weight=0.7):
    query = {
        "size": 10,
        "_source": {"exclude": ["vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [keyword_weight, semantic_weight]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

# 검색결과 비교하기

이전 단계에서 수행한 키워드 검색, 시맨틱 검색과 동일한 쿼리로 검색하여 결과를 비교해봅니다.


In [ ]:
query_text = "어벤져스와 비슷한 액션 SF 추천해줘"

In [ ]:
keyword_search(query_text, index_name, aos_client)

In [ ]:
semantic_search(query_text, index_name, aos_client, model_id)

In [ ]:
hybrid_search(query_text, keyword_weight=0.5, semantic_weight=0.5)

## Clean Up


In [ ]:
# aos_client.indices.delete(index=index_name)